## Centrality Dependence of CNM Effects

In [7]:
# --- setup (nPDF only) ---
import os, sys, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
sys.path.append("./npdf_code")

from npdf_data   import NPDFSystem, RpAAnalysis
from gluon_ratio import EPPS21Ratio, GluonEPPSProvider
from glauber     import OpticalGlauber, SystemSpec

# I/O
pPb5_dir = "./input/npdf/pPb5TeV"
pPb8_dir = "./input/npdf/pPb8TeV"
epps_dir = "./input/npdf/nPDFs"
outdir   = Path("./output-npdf-test"); outdir.mkdir(exist_ok=True)

# energies & σ_NN (mb)   (mb → fm² via ×0.1 inside the Glauber weight)
SQRTS  = {"5.02": 5023.0, "8.16": 8160.0}
SIG_NN  = { "5.02": 67.0,   "8.16": 71.0 }
CENT_EDGES = [0,20,40,60,80,100]    # % bins for centrality

# y windows you care about
Y_WINDOWS = [(-4.46,-2.96,"Backward"), (-1.37,0.43,"Mid"), (2.03,3.53,"Forward")]

# Style (no grid, no title; legend inside; 5.02 dashed, 8.16 solid)
COL = {"5.02":"tab:blue", "8.16":"tab:red"}
LS  = {"5.02":"--",      "8.16":"-"}
plt.rcParams.update({"figure.dpi":150, "font.size":12})

# Only helper we allow (Hessian symmetric 90% CL band)
def hessian_sym_90(members):
    M = np.asarray(members, float)
    if M.ndim == 1: M = M.reshape(48,1)
    out = np.zeros(M.shape[1], float)
    for i in range(0, M.shape[0], 2):
        d = M[i] - M[i+1]
        out += d*d
    return 0.5*np.sqrt(out)  # 90% CL; divide by 1.645 for ~68%

In [8]:
# ---- 5.02 TeV ----
ana  = RpAAnalysis()
sys5 = NPDFSystem.from_folder(pPb5_dir, kick="pp")
r5 = ana.compute_rpa_grid(sys5.df_pp, sys5.df_pa, sys5.df_errors, join="intersect", include_members=True)

if isinstance(r5, tuple) and len(r5)>=3:
    base5, r0c5, mem5 = r5[0], np.asarray(r5[1], float), np.asarray(r5[2], float)  # (48,N)
    y5  = base5["y"].to_numpy(float); pt5 = base5["pt"].to_numpy(float)
    R0_5sets = np.vstack([r0c5[None,:], mem5])   # (49, N)
else:
    df = r5 if isinstance(r5, pd.DataFrame) else pd.DataFrame(r5)
    y5  = df["y"].to_numpy(float); pt5 = df["pt"].to_numpy(float)
    memcols = [c for c in df.columns if c.lower().startswith("m")]
    memcols = sorted(memcols, key=lambda c: int(''.join(ch for ch in c if ch.isdigit()) or 0))
    R0_5sets = np.vstack([df["r_central"].to_numpy(float)[None,:],
                          df[memcols].to_numpy(float).T])  # (49,N)

# ---- 8.16 TeV ----
ana8 = RpAAnalysis()
sys8 = NPDFSystem.from_folder(pPb8_dir, kick="pp")
# r8   = ana8.compute_rpa_grid(sys8.df_pp, sys8.df_pa, sys8.df_errors, join="intersect")
r8 = ana.compute_rpa_grid(sys8.df_pp, sys8.df_pa, sys8.df_errors, join="intersect", include_members=True)

if isinstance(r8, tuple) and len(r8)>=3:
    base8, r0c8, mem8 = r8[0], np.asarray(r8[1], float), np.asarray(r8[2], float)
    y8  = base8["y"].to_numpy(float); pt8 = base8["pt"].to_numpy(float)
    R0_8sets = np.vstack([r0c8[None,:], mem8])
else:
    df = r8 if isinstance(r8, pd.DataFrame) else pd.DataFrame(r8)
    y8  = df["y"].to_numpy(float); pt8 = df["pt"].to_numpy(float)
    memcols = [c for c in df.columns if c.lower().startswith("m")]
    memcols = sorted(memcols, key=lambda c: int(''.join(ch for ch in c if ch.isdigit()) or 0))
    R0_8sets = np.vstack([df["r_central"].to_numpy(float)[None,:],
                          df[memcols].to_numpy(float).T])

In [9]:
# Analysis windows for this pass
ywin = (-4.46, -2.96)       # pick one of your windows first; we’ll loop later
ptwin= (0.0, 25.0)

# Build a modest mesh to average over (kept small for speed/learning)
y_grid  = np.linspace(ywin[0], ywin[1], 31)
pt_grid = np.linspace(ptwin[0], ptwin[1], 31)
Yg, PTg = np.meshgrid(y_grid, pt_grid, indexing="ij")
yv, pv  = Yg.ravel(), PTg.ravel()              # Nmesh = 31*31

# nearest-neighbor indices into the R0 tables
def _nn_index(ytable, ptable, yv, pv):
    return np.array([np.argmin((ytable-yi)**2 + (ptable-pi)**2) for yi,pi in zip(yv,pv)], int)

idx5 = _nn_index(y5, pt5, yv, pv)   # length Nmesh
idx8 = _nn_index(y8, pt8, yv, pv)

In [10]:
# Providers
prov5 = GluonEPPSProvider(EPPS21Ratio(A=208, path=epps_dir),
                          sqrt_sNN_GeV=SQRTS["5.02"],
                          m_state_GeV="charmonium", y_sign_for_xA=-1).with_geometry(None)
prov8 = GluonEPPSProvider(EPPS21Ratio(A=208, path=epps_dir),
                          sqrt_sNN_GeV=SQRTS["8.16"],
                          m_state_GeV="charmonium", y_sign_for_xA=-1).with_geometry(None)

# For each energy: build K0, Kf as (49, Nmesh) using the NN points (yv,pv)
K0_5 = np.empty((49, yv.size), float); Kf_5 = np.empty_like(K0_5)
K0_8 = np.empty((49, yv.size), float); Kf_8 = np.empty_like(K0_8)

for sid in range(1,50):
    # 5.02
    SA5  = prov5.SA_ypt_set(yv, pv, set_id=sid, flav="g")
    SA05 = prov5.SAWS_ypt_b_set(yv, pv, 0.0,  set_id=sid, flav="g")
    SAf5 = prov5.SAWS_ypt_b_set(yv, pv, 12.0, set_id=sid, flav="g")
    K0_5[sid-1] = SA05/np.clip(SA5,1e-12,None)
    Kf_5[sid-1] = SAf5/np.clip(SA5,1e-12,None)

    # 8.16
    SA8  = prov8.SA_ypt_set(yv, pv, set_id=sid, flav="g")
    SA08 = prov8.SAWS_ypt_b_set(yv, pv, 0.0,  set_id=sid, flav="g")
    SAf8 = prov8.SAWS_ypt_b_set(yv, pv, 12.0, set_id=sid, flav="g")
    K0_8[sid-1] = SA08/np.clip(SA8,1e-12,None)
    Kf_8[sid-1] = SAf8/np.clip(SA8,1e-12,None)

In [11]:
def _glauber_pack(energy_label):
    spec = SystemSpec(system="pA", roots_GeV=SQRTS[energy_label], A=208, sigma_nn_mb=SIG_NN[energy_label])
    gl   = OpticalGlauber(spec, verbose=False)

    b   = gl.b_grid.astype(float)             # fm
    TpA = gl.TpA_b.astype(float)              # pA inelastic thickness vs b
    TA  = np.asarray(gl.TA_r(b), float)       # A thickness vs b
    T0  = float(TA[0])

    # alpha(b) and inelastic weight (2π b [1-exp(-σNN*T_pA)])
    alpha_b = TA/T0
    w_b     = 2.0*np.pi*b*(1.0 - np.exp(-0.1*SIG_NN[energy_label]*TpA))

    # CDF to invert centrality (%)->b
    cdf = np.r_[0.0, np.cumsum(0.5*(w_b[1:]+w_b[:-1])*np.diff(b))]
    cdf = cdf/(cdf[-1] if cdf[-1]>0 else 1.0)

    def b_at(p):
        j = np.searchsorted(cdf, p, side="left")
        if j<=0: return b[0]
        if j>=len(b): return b[-1]
        t = 0.0 if cdf[j]==cdf[j-1] else (p - cdf[j-1])/(cdf[j]-cdf[j-1])
        return (1.0-t)*b[j-1] + t*b[j]

    # centrality bin masks in b and <b> per bin
    masks = []
    b_avg = []
    for L, R in zip(CENT_EDGES[:-1], CENT_EDGES[1:]):
        bL, bR = b_at(L/100.0), b_at(R/100.0)
        m = (b>=bL)&(b<=bR)
        masks.append(m)
        den = np.trapz(w_b[m], b[m])
        num = np.trapz(b[m]*w_b[m], b[m])
        b_avg.append( num/den if den>0 else np.nan )

    return dict(b=b, alpha_b=alpha_b, w_b=w_b, masks=masks, b_avg=np.array(b_avg))

G5 = _glauber_pack("5.02")
G8 = _glauber_pack("8.16")

print("⟨b⟩ (fm) 5.02:", np.round(G5["b_avg"],3))
print("⟨b⟩ (fm) 8.16:", np.round(G8["b_avg"],3))

/tmp/ipykernel_189499/4048623788.py:32: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  den = np.trapz(w_b[m], b[m])
/tmp/ipykernel_189499/4048623788.py:33: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  num = np.trapz(b[m]*w_b[m], b[m])


⟨b⟩ (fm) 5.02: [2.268 4.239 5.515 6.552 7.747]
⟨b⟩ (fm) 8.16: [2.334 4.284 5.515 6.553 7.752]


In [12]:
def alpha_bar_bins(energy_label: str, cent_edges):
    # geometry
    roots = 5023.0 if energy_label == "5.02" else 8160.0
    spec  = SystemSpec(system="pA", roots_GeV=roots, A=208,
                       sigma_nn_mb=SIG_NN_MB[energy_label])
    gl    = OpticalGlauber(spec, verbose=False)
    b, T  = gl.b_grid, gl.TpA_b
    s     = SIG_NN_MB[energy_label] * 0.1  # mb -> fm^2
    # event PDF and *collisional* weight
    w_evt   = 2.0*np.pi*b*(1.0 - np.exp(-s*T))          # P_inel(b) density
    w_coll  = w_evt * T                                 # ∝ P_inel × N_coll
    # centrality edges from event CDF (how experiments cut centrality)
    cdf_evt = np.r_[0.0, np.cumsum(0.5*(w_evt[1:]+w_evt[:-1]) * np.diff(b))]
    cdf_evt /= cdf_evt[-1] if cdf_evt[-1] > 0 else 1.0
    def b_from_percent(p):
        return float(np.interp(p/100.0, cdf_evt, b))
    T0 = float(T[0]) if T[0] > 0 else 1.0

    out = []
    for L, R in zip(cent_edges[:-1], cent_edges[1:]):
        bL, bR = b_from_percent(L), b_from_percent(R)
        m = (b >= bL) & (b < bR)
        den = np.trapezoid(w_coll[m], b[m])
        num = np.trapezoid((T[m]/T0) * w_coll[m], b[m])
        out.append(num/den if den > 0 else np.nan)
    return np.asarray(out, float)

In [13]:
# R0 meshes (central + 48 members picked by our NN indices)
R0m_5 = R0_5sets[:, idx5]     # (49, Nmesh)
R0m_8 = R0_8sets[:, idx8]     # (49, Nmesh)

# Broadcast α(b)
ab5 = G5["alpha_b"][:, None]  # (Nb,1)
ab8 = G8["alpha_b"][:, None]

# Build K(b) on the fly from anchors and α(b), then R = R0 * K (all by broadcasting)
# Shapes: Kbar → (49, Nb, Nmesh); Rraw → (49, Nb, Nmesh)
Kbar_5 = (Kf_5[:,None,:] + (K0_5[:,None,:] - Kf_5[:,None,:]) * ab5)
Kbar_8 = (Kf_8[:,None,:] + (K0_8[:,None,:] - Kf_8[:,None,:]) * ab8)

Rraw_5 = R0m_5[:,None,:] * Kbar_5
Rraw_8 = R0m_8[:,None,:] * Kbar_8

# For later convenience keep shapes explicit
Nb5, Nb8 = ab5.shape[0], ab8.shape[0]
Ny, Np   = len(y_grid), len(pt_grid)

In [14]:
# --- choose weights ---
# y,pT uniform weights (vector of length Nmesh)
W_yp = np.ones(yv.size, float)
W_yp /= W_yp.sum()

# centrality inelastic weights per bin come from masks over w_b
w5 = G5["w_b"]; b5 = G5["b"]; M5 = G5["masks"]
w8 = G8["w_b"]; b8 = G8["b"]; M8 = G8["masks"]

# --- average over y,pT first → profile vs b ---
# shape: (49, Nb)
Rprof_b_5 = np.tensordot(Rraw_5, W_yp, axes=([2],[0]))  # sum over Nmesh
Rprof_b_8 = np.tensordot(Rraw_8, W_yp, axes=([2],[0]))

# --- then bin over b using inelastic weights ---
Rsets_5 = []
Rsets_8 = []
for m in M5:
    den = np.trapezoid(w5[m], b5[m])
    num = np.trapezoid(Rprof_b_5[:, m] * w5[m], b5[m], axis=1)  # (49,)
    Rsets_5.append( num/den )

for m in M8:
    den = np.trapezoid(w8[m], b8[m])
    num = np.trapezoid(Rprof_b_8[:, m] * w8[m], b8[m], axis=1)
    Rsets_8.append( num/den )

Rsets_5 = np.stack(Rsets_5, axis=1)  # (49, Ncent)
Rsets_8 = np.stack(Rsets_8, axis=1)
cent_mid = 0.5*(np.array(CENT_EDGES[:-1])+np.array(CENT_EDGES[1:]))

In [15]:
# OPTIONAL: swap uniform W_yp with σ_pA weights (central, same energy)

# --- build quick σ_pp and σ_pA samplers on the fly (nearest-neighbor) ---
dfpp5, dfpa5 = sys5.df_pp, sys5.df_pa
Ypp5, Ppp5, Spp5 = dfpp5["y"].to_numpy(), dfpp5["pt"].to_numpy(), dfpp5["val"].to_numpy()
Ypa5, Ppa5, Spa5 = dfpa5["y"].to_numpy(), dfpa5["pt"].to_numpy(), dfpa5["val"].to_numpy()

idx_pp5 = np.array([np.argmin((Ypp5-yi)**2 + (Ppp5-pi)**2) for yi,pi in zip(yv,pv)], int)
idx_pa5 = np.array([np.argmin((Ypa5-yi)**2 + (Ppa5-pi)**2) for yi,pi in zip(yv,pv)], int)

W_yp_pa5 = Spa5[idx_pa5].astype(float)
W_yp_pa5 = np.clip(W_yp_pa5, 0.0, None)
W_yp_pa5 /= W_yp_pa5.sum()

# redo only these two lines if you want σ_pA weighting for 5 TeV
Rprof_b_5 = np.tensordot(Rraw_5, W_yp_pa5, axes=([2],[0]))

In [16]:
# build once per energy
s5   = SIG_NN_MB["5.02"] * 0.1
w5_e = 2*np.pi*b5*(1.0 - np.exp(-s5*T5))
w5_c = w5_e * T5

s8   = SIG_NN_MB["8.16"] * 0.1
w8_e = 2*np.pi*b8*(1.0 - np.exp(-s8*T8))
w8_c = w8_e * T8
# --- Min-bias (0–100%) = all b  (use np.trapezoid) ---
# Min-bias (0–100%) with collisional weight
den5_mb      = np.trapezoid(w5_c, b5)
Rc5_mb_sets  = np.trapezoid(Rprof_b_5 * w5_c, b5, axis=1) / den5_mb

den8_mb      = np.trapezoid(w8_c, b8)
Rc8_mb_sets  = np.trapezoid(Rprof_b_8 * w8_c, b8, axis=1) / den8_mb

Rc5_mb  = float(Rc5_mb_sets[0])
band5_mb = float(np.squeeze(hessian_sym_90(Rc5_mb_sets[1:])))
Rc8_mb  = float(Rc8_mb_sets[0])
band8_mb = float(np.squeeze(hessian_sym_90(Rc8_mb_sets[1:])))

# iterate over TRIPLES (yL, yR, name)
WINS = [(-4.46,-2.96,"Backward"), (-1.37,0.43,"Mid"), (2.03,3.53,"Forward")]

NameError: name 'T5' is not defined

In [ ]:
# shapes
print("Rsets_5 shape:", Rsets_5.shape)  # (49, Ncent)
print("Rsets_8 shape:", Rsets_8.shape)

# MB closure test (optional)
den5 = np.trapezoid(w5, b5)
mb5_from_bins = np.average(Rsets_5[0], weights=np.diff(np.asarray(CENT_EDGES))/100.0)  # crude width-weighted
mb5_direct    = Rc5_mb
print("MB bins vs direct (5.02):", mb5_from_bins, mb5_direct)


In [ ]:
## FIgure
fig, axs = plt.subplots(1, 3, figsize=(12.5,3.4), sharey=True)
for ax, (yL,yR,name) in zip(axs, WINS):
    ax.grid(False)

    # EASIEST: draw MB bands with axhspan (no shape headaches)
    ax.axhspan(Rc5_mb - band5_mb, Rc5_mb + band5_mb, color=COL["5.02"], alpha=0.12)
    ax.axhspan(Rc8_mb - band8_mb, Rc8_mb + band8_mb, color=COL["8.16"], alpha=0.12)
    ax.hlines(Rc5_mb, 0, 100, colors=COL["5.02"], linestyles=LS["5.02"], lw=1.0)
    ax.hlines(Rc8_mb, 0, 100, colors=COL["8.16"], linestyles=LS["8.16"], lw=1.0)

    # NOTE: Rc5/Rc8 below must correspond to THIS (yL,yR) window.
    # If you only computed one window upstream, recompute Rsets_5/8 for (yL,yR) first.
    Rc5  = Rsets_5[0]; band5 = hessian_sym_90(Rsets_5[1:])
    Rc8  = Rsets_8[0]; band8 = hessian_sym_90(Rsets_8[1:])
    step_x = np.r_[CENT_EDGES[:-1], CENT_EDGES[-1]]

    ax.step(step_x, np.r_[Rc5, Rc5[-1]], where="post",
            color=COL["5.02"], ls=LS["5.02"], lw=2.2, label="5.02 TeV")
    ax.fill_between(step_x, np.r_[Rc5-band5, (Rc5-band5)[-1]],
                            np.r_[Rc5+band5, (Rc5+band5)[-1]],
                    step="post", alpha=0.22, color=COL["5.02"])

    ax.step(step_x, np.r_[Rc8, Rc8[-1]], where="post",
            color=COL["8.16"], ls=LS["8.16"], lw=2.2, label="8.16 TeV")
    ax.fill_between(step_x, np.r_[Rc8-band8, (Rc8-band8)[-1]],
                            np.r_[Rc8+band8, (Rc8+band8)[-1]],
                    step="post", alpha=0.22, color=COL["8.16"])

    ax.axhline(1.0, ls=":", c="k", lw=1.0)
    ax.set_xlim(0,100); ax.set_ylim(0.35,1.25)
    ax.set_xlabel("Centrality [%]")
    ax.text(0.02, 0.95, f"{yL:g} < y < {yR:g}\n$p_T\\in[{ptwin[0]:g},{ptwin[1]:g}]$ GeV",
            transform=ax.transAxes, va="top", ha="left", fontsize=10,
            bbox=dict(facecolor="white", alpha=0.75, edgecolor="none"))

axs[0].set_ylabel(r"$R_{pA}$")
axs[-1].legend(frameon=False, loc="lower right")
fig.tight_layout()
fig.savefig(outdir/"rpa_vs_centrality_3panels_rawband.png")
fig

In [ ]:
PT_WIN=(0,20)

In [ ]:
COL = {"5.02":"tab:blue","8.16":"tab:red"}
LS  = {"5.02":"--", "8.16":"-"}   # dashed=5, solid=8 as you asked

results = {}  # stash per-window for plotting later

for (yL,yR,name) in Y_WINDOWS:
    # --- select (y,pT) points in window for both energies (use their own grids) ---
    m5 = (r5["y"].between(yL,yR)) & (r5["pt"].between(PT_WIN[0], PT_WIN[1]))
    g5 = r5.loc[m5].reset_index(drop=True)
    m8 = (r8["y"].between(yL,yR)) & (r8["pt"].between(PT_WIN[0], PT_WIN[1]))
    g8 = r8.loc[m8].reset_index(drop=True)

    # --- extract central + 48 members (49 × Npoints), keep the same member ordering ---
    def _R0_sets(g):
        cols = sorted([c for c in g.columns if c.startswith("r_mem_")],
                      key=lambda s: int(s.split("_")[-1]))
        R0 = np.vstack([g["r_central"].to_numpy(float)[None,:],
                        g[cols].to_numpy(float).T])
        YP = g[["y","pt"]].to_numpy(float)
        return YP, R0

    YP5, R0_5 = _R0_sets(g5)
    YP8, R0_8 = _R0_sets(g8)

    # --- K anchors per set (vectorized in (y,pT), loop over set_id only) ---
    def _K_anchors(provider, YP):
        yv, pv = YP[:,0], YP[:,1]
        K0 = np.empty((49, len(YP)), float)
        Kf = np.empty_like(K0)
        for sid in range(1,50):
            SA  = provider.SA_ypt_set(yv, pv, set_id=sid, flav="g")
            K0[sid-1] = provider.SAWS_ypt_b_set(yv, pv, 0.0,  set_id=sid, flav="g") / np.clip(SA, 1e-12, None)
            Kf[sid-1] = provider.SAWS_ypt_b_set(yv, pv, 12.0, set_id=sid, flav="g") / np.clip(SA, 1e-12, None)
        return K0, Kf

    K0_5, Kf_5 = _K_anchors(prov5, YP5)
    K0_8, Kf_8 = _K_anchors(prov8, YP8)

    # --- σ_pA weights in (y,pT) from the *central* set of the same energy ---
    def _pa_weights(sys, YP):
        df = sys.df_pa[["y","pt","val"]]  # NOTE: 'val', not 'sigma'
        A  = df[["y","pt"]].to_numpy(float)
        W  = np.empty(len(YP), float)
        for i,(yy,pp) in enumerate(YP):
            j = int(np.argmin((A[:,0]-yy)**2 + (A[:,1]-pp)**2))
            W[i] = max(df["val"].iloc[j], 0.0)
        s = W.sum()
        return W/s if s>0 else np.full_like(W, 1.0/len(W))

    Wyp5 = _pa_weights(sys5, YP5)
    Wyp8 = _pa_weights(sys8, YP8)

    # --- ᾱ per centrality bin (uses the SAME provider you used for K) ---
    abar5 = alpha_bar_bins("5.02", prov5, CENT_EDGES)
    abar8 = alpha_bar_bins("8.16", prov8, CENT_EDGES)

    cent_mid = 0.5*(np.asarray(CENT_EDGES[:-1])+np.asarray(CENT_EDGES[1:]))

    # --- centrality-binned RpA for all sets, then 90% Hessian band ---
    Rsets5 = []
    for a in abar5:
        Kbar = Kf_5 + (K0_5 - Kf_5)*a            # (49,Npoints)
        Rk   = (R0_5 * Kbar) @ Wyp5              # (49,)
        Rsets5.append(Rk)
    Rsets5 = np.stack(Rsets5, axis=1)            # (49,Ncent)
    Rc5    = Rsets5[0]
    band5  = hessian_sym_90(Rsets5[1:])          # (Ncent,)

    Rsets8 = []
    for a in abar8:
        Kbar = Kf_8 + (K0_8 - Kf_8)*a
        Rk   = (R0_8 * Kbar) @ Wyp8
        Rsets8.append(Rk)
    Rsets8 = np.stack(Rsets8, axis=1)
    Rc8    = Rsets8[0]
    band8  = hessian_sym_90(Rsets8[1:])

    # --- Min-bias bands (0–100%) as horizontal bands we’ll draw behind the steps ---
    a5_mb  = alpha_bar_bins("5.02", prov5, [0,100])[0]
    a8_mb  = alpha_bar_bins("8.16", prov8, [0,100])[0]
    Rc5_mb = (R0_5 * (Kf_5 + (K0_5 - Kf_5)*a5_mb)) @ Wyp5
    Rc8_mb = (R0_8 * (Kf_8 + (K0_8 - Kf_8)*a8_mb)) @ Wyp8
    band5_mb = hessian_sym_90(((R0_5 * (Kf_5 + (K0_5 - Kf_5)*a5_mb)) @ Wyp5)[None,:] * 0 + Rsets5[1:, :].mean(axis=1)*0 + 0)  # zero → use Rc5_mb only
    band8_mb = hessian_sym_90(((R0_8 * (Kf_8 + (K0_8 - Kf_8)*a8_mb)) @ Wyp8)[None,:] * 0 + Rsets8[1:, :].mean(axis=1)*0 + 0)

    # stash everything
    results[name] = dict(cent_mid=cent_mid, Rc5=Rc5, band5=band5,
                         Rc8=Rc8, band8=band8, Rc5_mb=Rc5_mb, Rc8_mb=Rc8_mb)
